1. Write a function that converts number representation (bin<->dec<->hex)

In [96]:
def bin_to_dec(k):
        k1=k
        m=1
        if k1[0]=='-':
            k1=k1[1:]
            m=m*(-1)
        b=0
        k1=k1[2:]
        ln=len(k1)
        for i in range(ln):
            o=int(k1[i])*2**(ln-1-i)
            b=b+o
        d=b
        if m==-1:
            d=d*(-1)
            
        return ('Decimal representation of {} is {}'.format(k,d))
def hex_to_dec(k):
    bbb=k
    m=1
    if k[0]=='-':
        k=k[1:]
        m=m*(-1)
    dic={'a':10,'b':11,'c':12,'d':13,'e':14,'f':15}
    b=0
    k=k[2:]
    ln=len(k)
    for i in range(ln):
            if k[i] in dic:
                o=dic[k[i]]*16**(ln-1-i)
            else:
                o=int(k[i])*16**(ln-1-i)
            b=b+o
    d=b
    if m==-1:
        d=d*(-1)
    return ('Decimal representation of {} is {}'.format(bbb,d))
def dec_to_BinHex(k):  
            k=int(k)
            if k<0:
                k1=-1*k
                k2=-1*k
            else:
                k1=k
                k2=k
            m=[]
            while k1>0:
                    l=k1%2
                    m.append(l)
                    k1=int(k1/2)
            b=0
            for i in range(len(m)):
                o=m[i]*10**(i)
                b=b+o

            c=str(b)
            if k<0:
                    d1="-0b"+c
            else:
                    d1="0b"+c
            n=[]
            dic2={'10':'a','11':'b','12':'c','13':'d','14':'e','15':'f'}
            while k2>0:
                    ll=k2%16
                    if ll in range(10,16):
                        n.append(dic2[str(ll)])
                    else:
                        n.append(str(ll))
                    k2=int(k2/16)
            n.reverse()
            o=''
            for i in range(len(n)):
                        o+=n[i]


            if k<0:
                    dd1="-0x"+o
            else:
                    dd1="0x"+o

            d=(d1,dd1)
            return ('Binary representation of {} is {} and Hexadecimal representation of {} is {}' .format(k,d1,k,dd1))


def conv():
        k=input("write the number:")
    
        if  (len(k)>1 and k[1]=="b") or (len(k)>2 and k[2]=='b'):
             print(bin_to_dec(k))
        elif (len(k)>1 and k[1]=="x") or (len(k)>2 and k[2]=='x'):
             print(hex_to_dec(k))
        else:
             print(dec_to_BinHex(k))
   

conv()

write the number:-197
Binary representation of -197 is -0b11000101 and Hexadecimal representation of -197 is -0xc5


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [2]:
def bin_to_dec(k):
        b=0
        k1=k
        ln=len(k1)
        for i in range(ln):
            o=int(k1[i])*2**(ln-1-i)
            b=b+o
        d=b
        return d
def spfp():
    k=input("write the word:")
    k=k[::-1]
    mantissa=1
    for i in range(23):
        d=(2**((-1-i))*int(k[23-1-i]))
        mantissa=mantissa+d
    k1=k[23:31]
    k1=k1[::-1]
    b=bin_to_dec(k1)-127
    s=(-1)**int(k[31])
    x=s*(2**b)*mantissa
    return x
print(spfp())
    
    

write the word:00000011110101010000101010101000
1.2521461391272196e-36


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [5]:
d=1
c=1
n=1080
for i in range(n):
    c=c/2
    print(i,"%10.3e"%(c))
print('\n')
#c=4.941e-324 is the smallest number
for i in range(n):
    d=d*2
    print(i,"%10.3e"%(d))
#d=8.988e+307 is the biggest number    

0  5.000e-01
1  2.500e-01
2  1.250e-01
3  6.250e-02
4  3.125e-02
5  1.562e-02
6  7.812e-03
7  3.906e-03
8  1.953e-03
9  9.766e-04
10  4.883e-04
11  2.441e-04
12  1.221e-04
13  6.104e-05
14  3.052e-05
15  1.526e-05
16  7.629e-06
17  3.815e-06
18  1.907e-06
19  9.537e-07
20  4.768e-07
21  2.384e-07
22  1.192e-07
23  5.960e-08
24  2.980e-08
25  1.490e-08
26  7.451e-09
27  3.725e-09
28  1.863e-09
29  9.313e-10
30  4.657e-10
31  2.328e-10
32  1.164e-10
33  5.821e-11
34  2.910e-11
35  1.455e-11
36  7.276e-12
37  3.638e-12
38  1.819e-12
39  9.095e-13
40  4.547e-13
41  2.274e-13
42  1.137e-13
43  5.684e-14
44  2.842e-14
45  1.421e-14
46  7.105e-15
47  3.553e-15
48  1.776e-15
49  8.882e-16
50  4.441e-16
51  2.220e-16
52  1.110e-16
53  5.551e-17
54  2.776e-17
55  1.388e-17
56  6.939e-18
57  3.469e-18
58  1.735e-18
59  8.674e-19
60  4.337e-19
61  2.168e-19
62  1.084e-19
63  5.421e-20
64  2.711e-20
65  1.355e-20
66  6.776e-21
67  3.388e-21
68  1.694e-21
69  8.470e-22
70  4.235e-22
71  2.118e-22
72

480 6.243e+144
481 1.249e+145
482 2.497e+145
483 4.995e+145
484 9.990e+145
485 1.998e+146
486 3.996e+146
487 7.992e+146
488 1.598e+147
489 3.197e+147
490 6.393e+147
491 1.279e+148
492 2.557e+148
493 5.115e+148
494 1.023e+149
495 2.046e+149
496 4.092e+149
497 8.183e+149
498 1.637e+150
499 3.273e+150
500 6.547e+150
501 1.309e+151
502 2.619e+151
503 5.237e+151
504 1.047e+152
505 2.095e+152
506 4.190e+152
507 8.380e+152
508 1.676e+153
509 3.352e+153
510 6.704e+153
511 1.341e+154
512 2.682e+154
513 5.363e+154
514 1.073e+155
515 2.145e+155
516 4.290e+155
517 8.581e+155
518 1.716e+156
519 3.432e+156
520 6.865e+156
521 1.373e+157
522 2.746e+157
523 5.492e+157
524 1.098e+158
525 2.197e+158
526 4.393e+158
527 8.787e+158
528 1.757e+159
529 3.515e+159
530 7.030e+159
531 1.406e+160
532 2.812e+160
533 5.624e+160
534 1.125e+161
535 2.249e+161
536 4.499e+161
537 8.998e+161
538 1.800e+162
539 3.599e+162
540 7.198e+162
541 1.440e+163
542 2.879e+163
543 5.759e+163
544 1.152e+164
545 2.303e+164
546 4.607e

OverflowError: int too large to convert to float

4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [35]:
eps=1.0
n=55
for i in range(n):
    eps=eps/2
    m=1.0+eps
    print(i,m,eps)
#from eps=2.220446049250313e-16, m will be 1 and eps goes to 0

0 1.5 0.5
1 1.25 0.25
2 1.125 0.125
3 1.0625 0.0625
4 1.03125 0.03125
5 1.015625 0.015625
6 1.0078125 0.0078125
7 1.00390625 0.00390625
8 1.001953125 0.001953125
9 1.0009765625 0.0009765625
10 1.00048828125 0.00048828125
11 1.000244140625 0.000244140625
12 1.0001220703125 0.0001220703125
13 1.00006103515625 6.103515625e-05
14 1.000030517578125 3.0517578125e-05
15 1.0000152587890625 1.52587890625e-05
16 1.0000076293945312 7.62939453125e-06
17 1.0000038146972656 3.814697265625e-06
18 1.0000019073486328 1.9073486328125e-06
19 1.0000009536743164 9.5367431640625e-07
20 1.0000004768371582 4.76837158203125e-07
21 1.000000238418579 2.384185791015625e-07
22 1.0000001192092896 1.1920928955078125e-07
23 1.0000000596046448 5.960464477539063e-08
24 1.0000000298023224 2.9802322387695312e-08
25 1.0000000149011612 1.4901161193847656e-08
26 1.0000000074505806 7.450580596923828e-09
27 1.0000000037252903 3.725290298461914e-09
28 1.0000000018626451 1.862645149230957e-09
29 1.0000000009313226 9.31322574615

5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [15]:
#1
def solut(a,b,c):
    
    x1=(-b+(((b**2)-(4*a*c))**(1/2)))/(2*a)
    x2=(-b-(((b**2)-(4*a*c))**(1/2)))/(2*a) 
    return(x1,x2)
a=0.001
b=1000
c=0.001
print(solut(a,b,c))

#2
def solut2(a,b,c):
    x1=((-b+(((b**2)-(4*a*c))**(1/2)))*((-b)-(((b**2)-(4*a*c))**(1/2))))/((2*a)*((-b)-(((b**2)-(4*a*c))**(1/2))))
    x2=((-b-(((b**2)-(4*a*c))**(1/2)))*((-b)+(((b**2)-(4*a*c))**(1/2))))/((2*a)*((-b)+(((b**2)-(4*a*c))**(1/2))))
    return(x1,x2)
a=0.001
b=1000
c=0.001
print(solut2(a,b,c))

def solut3(a,b,c): 
    
    if ((b**2)-(4*a*c))>0: 
        return "there is two real roots:",(-b+(((b**2)-(4*a*c))**(1/2)))/(2*a),(-b-(((b**2)-(4*a*c))**(1/2)))/(2*a) 
      
    elif (b**2-4*a*c)==0: 
        return("there is just one real root:",-b/(2*a)) 
         
    else:
        return("the complex roots are:",-b/(2*a), " + i",(((b**2)-(4*a*c))**(1/2)),",",-b/(2*a), " - i",(((b**2)-(4*a*c))**(1/2))) 
a=0.001
b=1000
c=0.001
print(solut3(a,b,c))
              
              
from sympy.solvers import solve
from sympy import Symbol
x = Symbol('x')
solve(0.001*x**2+1000*x+0.001, x)


(-9.999894245993346e-07, -999999.999999)
(-9.999894245993346e-07, -999999.9999990001)
('there is two real roots:', -9.999894245993346e-07, -999999.999999)


[-999999.999999000, -1.00000000000100e-6]

6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

because delta is not near to zero enough.

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

in $\delta = 10^{−8}$ the difference is the lowest but by decreasing sigma the difference will increase again because in very small number the machine has error in calculating.


In [30]:
def f(x):
    return x*(x-1)
x=1
m=[]
delta=[10**(-2),10**(-4),10**(-6),10**(-8),10**(-10),10**(-12),10**(-14)]
for i in range(len(delta)):
    d=(f(x+delta[i])-f(x))/delta[i]
    print("delta:",delta[i],"   derivative:",d)
    m.append(d)
def df(x):
    return 2*x-1
dd=df(1)
print("\nanswer for analytical derivative:",dd)
n=[(i-dd) for i in m]
#print("\nthe differences are:",n)
for x, y in zip(delta,n):
    print("\nthe difference for delta=",x, 'is:',y)

#a)because delta is not near to zero enough.

#b)in delta=10**(-8) the difference is the lowest but by decreasing sigma the difference will increase again because in very small number the machine has error in calculating.

delta: 0.01    derivative: 1.010000000000001
delta: 0.0001    derivative: 1.0000999999998899
delta: 1e-06    derivative: 1.0000009999177333
delta: 1e-08    derivative: 1.0000000039225287
delta: 1e-10    derivative: 1.000000082840371
delta: 1e-12    derivative: 1.0000889005833413
delta: 1e-14    derivative: 0.9992007221626509

answer for analytical derivative: 1

the difference for delta= 0.01 is: 0.010000000000000897

the difference for delta= 0.0001 is: 9.999999988985486e-05

the difference for delta= 1e-06 is: 9.99917733279787e-07

the difference for delta= 1e-08 is: 3.922528746258536e-09

the difference for delta= 1e-10 is: 8.284037100736441e-08

the difference for delta= 1e-12 is: 8.890058334132256e-05

the difference for delta= 1e-14 is: -0.0007992778373491216


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [34]:
import math
from math import sqrt,pi
import numpy as np
n=50000

h=2/n
x = np.linspace(-1,1,n)
i=0
for k in x:
         i=i+h*sqrt(1-k**2)
print(i)
print("the difference:",i-pi/2)  
%timeit i


1.5707647621157648
the difference: -3.1564679131790285e-05
17.3 ns ± 0.627 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In [32]:
#17.3 ns ± 0.627 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)

n=(1/17)*(10**9)
print('N can increase by:',n, 'to be run in less then a second')


N can increase by: 58823529.4117647 to be run in less then a second
